In [2]:
!pip install mediapipe
!pip install opencv-python

In [5]:
import mediapipe as mp
import cv2
from skimage.io import imread
import struct


import numpy as np
from matplotlib import pyplot as plt

In [57]:
# color camera Props
color_width = 1920
color_height = 1080

color_focal_length_x = 1065.25 #1062.91
color_focal_length_y = 1065.25 #1062.91

color_principal_point_x = 959.5 #955.88
color_principal_point_y = 539.5 #556.11

color_skew =  0.35

color_radial_distortion_k1 = 0.0186
color_radial_distortion_k2 = -0.0211
color_radial_distortion_k3 = 0.0193

color_tangential_distortion_p1 = 0.00191
color_tangential_distortion_p2 = -0.00221

# depth camera Props
depth_width = 512
depth_height = 424

depth_focal_length_x = 364.2 #363.88
depth_focal_length_y = 364.2 #363.88

depth_principal_point_x = 254.878 #257.63
depth_principal_point_y = 205.395 #216.40

depth_skew =  0.01

depth_radial_distortion_k1 = 0.0948
depth_radial_distortion_k2 = -0.254
depth_radial_distortion_k3 = 0.0734

depth_tangential_distortion_p1 = 0.000146
depth_tangential_distortion_p2 = -0.000865

#extrinsic param

roation_matrix = [[0.99997, 0.00715, -0.00105],
                  [-0.00715, 0.99995, 0.00662],
                  [0.00110, -0.00661, 0.99998]]

translation_matrix = [0.0389, 0.00556, 0.02714]

# Populate color_intrinsics dictionary
color_intrinsics = {
    "width": color_width,
    "height": color_height,
    "fx": color_focal_length_x,
    "fy": color_focal_length_y,
    "cx": color_principal_point_x,
    "cy": color_principal_point_y,
    "k1": color_radial_distortion_k1,
    "k2": color_radial_distortion_k2,
    "k3": color_radial_distortion_k3,
    "p1": color_tangential_distortion_p1,
    "p2": color_tangential_distortion_p2,
}

# Populate depth_intrinsics dictionary
depth_intrinsics = {
    "width": depth_width,
    "height": depth_height,
    "fx": depth_focal_length_x,
    "fy": depth_focal_length_y,
    "cx": depth_principal_point_x,
    "cy": depth_principal_point_y,
    "k1": depth_radial_distortion_k1,
    "k2": depth_radial_distortion_k2,
    "k3": depth_radial_distortion_k3,
    "p1": depth_tangential_distortion_p1,
    "p2": depth_tangential_distortion_p2,
}

# Populate depth_to_color_extrinsics dictionary
depth_to_color_extrinsics = {
    "rotation_matrix": roation_matrix,
    "translation_vector": translation_matrix
}


# function that realigns the depth image to the colour image
def remap_depth_to_color(depth_image, color_intrinsics, depth_intrinsics, depth_to_color_extrinsics):
    # Create the camera matrices for color and depth cameras
    color_camera_matrix = np.array([
        [color_intrinsics["fx"], 0, color_intrinsics["cx"]],
        [0, color_intrinsics["fy"], color_intrinsics["cy"]],
        [0, 0, 1]
    ])


    depth_camera_matrix = np.array([
        [depth_intrinsics["fx"], 0, depth_intrinsics["cx"]],
        [0, depth_intrinsics["fy"], depth_intrinsics["cy"]],
        [0, 0, 1]
    ])



    color_distortion_coeffs = np.array([
        color_intrinsics["k1"],
        color_intrinsics["k2"],
        color_intrinsics["p1"],
        color_intrinsics["p2"],
        color_intrinsics["k3"],
    ])


    depth__distortion_coeffs = np.array([
        depth_intrinsics["k1"],
        depth_intrinsics["k2"],
        depth_intrinsics["p1"],
        depth_intrinsics["p2"],
        depth_intrinsics["k3"],
    ])


    color_distortion_coeffs = np.array([
        0.020429028189430772,
        -0.0078399448855923665,
       -0.0030949667668667908,
        0.0021848647826377197,
       -0.052344198896187882
        ])

    depth__distortion_coeffs = np.array([
        0.088866838848423224,
        -0.24742250848813654,
        -0.0028648128998355381,
        -0.00066909365375191258,
        0.057299947210392929
        ])


    # Get the rotation matrix and translation vector from the extrinsics
    R = np.eye(3)#np.array(depth_to_color_extrinsics["rotation_matrix"])

    new_camera_matrix, _ = cv2.getOptimalNewCameraMatrix(
        color_camera_matrix,
        color_distortion_coeffs,
        (1920, 1080),
        0
    )

    x_y_translation_matrix = np.array([
      [1, 0, (depth_to_color_extrinsics["translation_vector"][0]) * color_intrinsics["width"]],
      [0, 1, (depth_to_color_extrinsics["translation_vector"][1]) * color_intrinsics["height"]],
      [0, 0, 1]
    ], dtype=np.float32)


    center = (color_intrinsics["width"] // 2, color_intrinsics["height"] // 2)
    scaling_factor = (1 - depth_to_color_extrinsics["translation_vector"][2]) * 1.05 * 0.98425817 * 0.99389002
    x_y_scaling_matrix = np.array([
        [scaling_factor, 0, 0],
        [0, scaling_factor, 0],
        [0, 0, 1]
    ], dtype=np.float32)

    # Compute the transformation maps for remapping
    map_x, map_y = cv2.initUndistortRectifyMap(
        depth_camera_matrix,
        depth__distortion_coeffs,
        R,
        new_camera_matrix,
        (color_intrinsics["width"], color_intrinsics["height"]),
        cv2.CV_32FC1
    )

    # Remap the depth image to be the same size as the color image
    remapped_depth = cv2.remap(
        depth_image,
        map_x,
        map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=0
    )


    remapped_depth = cv2.warpPerspective(
        remapped_depth,
        x_y_scaling_matrix,
        (color_intrinsics["width"], color_intrinsics["height"]),
        cv2.INTER_LINEAR,
        cv2.BORDER_CONSTANT,
        0)

    remapped_depth = cv2.warpPerspective(
        remapped_depth,
        x_y_translation_matrix,
        (color_intrinsics["width"], color_intrinsics["height"]),
        cv2.INTER_LINEAR,
        cv2.BORDER_CONSTANT,
        0)


    #remapped_depth = apply_translation(color_intrinsics["width"], color_intrinsics["height"], remapped_depth, depth_to_color_extrinsics["translation_vector"])


    return remapped_depth


In [58]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)

img_width = 1920
img_height = 1080
dpt_width = 512
dpt_height = 424

In [59]:
import os
import csv

Project_Path = "C:\\Users\\Othmane Tazi\\Desktop\\PgProject"

Color_Path = os.path.join(Project_Path, 'KinectOutput/Images')
Depth_Path = os.path.join(Project_Path, 'KinectOutput/Depth')

In [60]:
# function to get the xyz values of the desired landmarks
def extract_hand_coords(Depth, Images, names):
    Coordinates = [ ]

    scale_x = dpt_width / img_width
    scale_y = dpt_height / img_height

    with mpHands.Hands(static_image_mode=False, max_num_hands=1) as hands:
        for img, dpt, nam in zip(Images, Depth, names):
            values = [[nam]]

            # Process the image and detect landmarks
            results = hands.process(img)

            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]

                # Define specific landmark desired
                landmark_indcies = [0, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20]

                for index in landmark_indcies:
                    # get desired landmark
                    landmark = hand_landmarks.landmark[index]

                    # Convert landmark coordinates from normalized [0, 1] to image coordinates
                    landmark_x = int(landmark.x * 1920)
                    landmark_y = int(landmark.y * 1080)
                    
                    # Get Depth Value associated with landmark
                    remapped_dpt = remap_depth_to_color(dpt, color_intrinsics, depth_intrinsics, depth_to_color_extrinsics)
                    landmark_z = remapped_dpt[landmark_y, landmark_x, 0]

                    # Append Landmark X, Y, Z to List
                    values.append([landmark_x, landmark_y, landmark_z])
                    

                # Add all landmark coordinates to Coordinates list
        
                Coordinates.append(values)
            
    return Coordinates

In [61]:
Colors_ = [ ]
Depths_ = [ ]

Coordinates = []

file_path = os.path.join(Project_Path, 'raw_coordinates.csv')

# Open the CSV file in write mode and create a csv.writer object
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    for x in range(0, 50):
        Colors_.clear()
        Depths_.clear()
        Coordinates.clear()

        n = x * 100
        for i in range(1 + n, 101 + n):
            color_img = imread(os.path.join(Color_Path, str(i) + '.png'))
            Colors_.append(color_img)

        for i in range(1 + n, 101 + n):
            depth_img = imread(os.path.join(Depth_Path, str(i) + '.png'))
            Depths_.append(depth_img)

        Coordinates = extract_hand_coords(Depths_, Colors_, range(1 + n, 101 + n))

        # write data to the CSV file
        for coordinate in Coordinates:
            flattened_list = [item for sublist in coordinate for item in sublist]
            writer.writerow(flattened_list)
            
        print(x)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
